In [2]:
import sys

import torch
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('./src/')
#sys.path.append('./rcwa_tf/src/')
sys.path.append('./rcwa_pt/src/')
import utils
print(torch.__version__)

1.10.1+cu113


In [3]:
# Limit GPU memory growth.
gpus = list_physical_devices("GPU")
print(gpus)
for gpu in gpus:
    set_memory_growth(gpu, True)

#ptDevice = torch.device('cuda:1')
#torch.cuda.set_per_process_memory_fraction(0.9, device=ptDevice)

# Choose the device to run on.
#use_gpu = True
##### REMEMBER: CHANGE BACK TO GPU:1
#tfDevice = '/job:localhost/replica:0/task:0/device:GPU:1' if use_gpu else '/CPU:0'
#print('Executing on device ' + tfDevice + ' ...')

# Measure GPU memory usage.
gpu_memory_init = tf_utils.gpu_memory_info()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
#ptDevice = torch.device('cuda:1')

In [5]:
cpu_memory_total, cpu_memory_used_init = tf_utils.cpu_memory_info()
gpu_memory_init = tf_utils.gpu_memory_info()

In [6]:
#import solver
#import solver_metasurface
import solver_pt
import solver_metasurface_pt

In [7]:
# Initialize parameters.
user_params = {}

# Flage to enable hyperparameter grid search.
user_params['enable_hyperparameter_gridsearch'] = False

# Tunable parameters.
user_params['N'] = 50
user_params['sigmoid_update'] = 20.0
user_params['learning_rate'] = 6E-1
user_params['initial_height'] = 3

user_params['parameter_string'] = 'N' + str(user_params['N']) \
    + '-sigmoid_update' + str(user_params['sigmoid_update']) \
    + '-learning_rate' + str(user_params['learning_rate']) \
    + '-initial_height' + str(user_params['initial_height'])

# Values to use in hyperparameter grid search.
# Stored as a dict. Each dict key is the key in user_params corresponding to a tunable
# hyperparameter, i.e. 'N' and its value is a list of values to try for that hyperparameter.
param_grid = {'N': [10],
              'sigmoid_update': [10.0, 20.0],
              'learning_rate': [2E-1],
              'initial_height': [0]}
user_params['param_grid'] = param_grid


# Source parameters.
user_params['wavelengths'] = [120000.0]
user_params['thetas'] = [0.0]
user_params['phis'] = [0.0]
user_params['pte'] = [1.0]
user_params['ptm'] = [0.0]

# Device parmeters.
user_params['pixelsX'] = 200
user_params['pixelsY'] = user_params['pixelsX']
user_params['erd'] = 12.04
user_params['ers'] = user_params['erd']
user_params['L'] = [50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 950000.0]
user_params['Lx'] = 20000.0
#user_params['Lx'] = 5000000. / user_params['pixelsX']
user_params['Ly'] = user_params['Lx']
user_params['f'] = 0.0 # Focal distance (nm)
#user_params['pt_device'] = ptDevice

# Solver parameters.
user_params['PQ'] = [3,3]
user_params['upsample'] = 7

# Problem parameters.
user_params['w_l1'] = 1.0
user_params['sigmoid_coeff'] = 1.0
user_params['focal_spot_radius'] = 10
user_params['enable_random_init'] = False
user_params['enable_debug'] = False
user_params['enable_print'] = True
user_params['enable_timing'] = True

# Logging parameters.
user_params['enable_logging'] = True
user_params['log_filename_prefix'] = 'nearfield-' + str(user_params['pixelsX']) + 'x' + str(user_params['pixelsY']) + '-'
user_params['log_filename_extension'] = '.log'
user_params['load_result'] = False
user_params['load_filename'] = user_params['log_filename_prefix'] + user_params['parameter_string'] + user_params['log_filename_extension']

In [8]:
def loss_function_pt(h, params):
    
    # Generate permittivity and permeability distributions.
    ER_t, UR_t = solver_metasurface_pt.generate_layered_metasurface(h, params)

    # Simulate the system.
    outputs = solver_pt.simulate_allsteps(ER_t, UR_t, params)
    
    # First loss term: maximize sum of electric field magnitude within some radius of the desired focal point.
    r = params['focal_spot_radius']
    field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
    focal_plane = solver_pt.propagate(params['input'] * field, params['propagator'], params['upsample'])
    index = (params['pixelsX'] * params['upsample']) // 2
    l1 = torch.sum(torch.abs(focal_plane[0, index-r:index+r, index-r:index+r]))

    # Final loss: (negative) field intensity at focal point + field intensity elsewhere.
    return -params['w_l1']*l1

In [9]:
# Set loss function.
user_params['loss_function'] = loss_function_pt

# Optimize.
h_pt, loss_pt, params_pt = solver_metasurface_pt.optimize_device(user_params)

# Get full device representation of resulting metasurface.
ER_t_pt, UR_t_pt = solver_metasurface_pt.generate_layered_metasurface(h_pt, params_pt)

# Simulate scattering from the optimized device.
outputs_pt = solver_pt.simulate_allsteps(ER_t_pt, UR_t_pt, params_pt)
field_pt = outputs_pt['ty'][:, :, :, np.prod(params_pt['PQ']) // 2, 0]
focal_plane_pt = solver_pt.propagate(params_pt['input'] * field_pt, params_pt['propagator'], params_pt['upsample'])

/home/deveringham/.local/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0, 

RuntimeError: CUDA out of memory. Tried to allocate 11.59 GiB (GPU 0; 47.54 GiB total capacity; 32.45 GiB already allocated; 10.63 GiB free; 34.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
intensity = torch.abs(focal_plane_pt[0, :, :]) ** 2
intensity = intensity.detach().cpu().numpy()
plt.imshow(intensity)
plt.colorbar()

In [ ]:
plt.plot(loss_pt)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
ER_t_pt, UR_t_pt = solver_metasurface_pt.generate_layered_metasurface(h_pt, params_pt)
solver_metasurface_pt.display_layered_metasurface(ER_t_pt, params_pt)

In [ ]:
gpu_memory_final = tf_utils.gpu_memory_info()
gpu_memory_used = [gpu_memory_final[1][0] - gpu_memory_init[1][0], gpu_memory_final[1][1] - gpu_memory_init[1][1]]
print('Memory used on each GPU(MiB): ' + str(gpu_memory_used))

_, cpu_memory_used_final = tf_utils.cpu_memory_info()
print('Memory used on the CPU(MB): ' + str(cpu_memory_used_final - cpu_memory_used_final))

In [ ]:
import json

with open("180x180_h_pt.txt", 'w', encoding="utf-8") as f:
        
    json.dump(h_pt.detach().cpu().numpy().tolist(), f)